In [1]:
import importlib

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

%matplotlib inline

In [2]:
model_module = importlib.import_module("handyrl.model")
env_module = importlib.import_module("handyrl.envs.kaggle.hungry_geese")

Loading environment football failed: No module named 'gfootball'


In [3]:
e = env_module.Environment()
e.reset()

In [4]:
obs = e.observation()
obs = torch.from_numpy(obs.reshape(1, 17, 7, 11)).clone()
obs_ = torch.cat([obs, obs], dim=0)
print(f"size: {obs_.size()}, type: {obs_.dtype}")

size: torch.Size([2, 17, 7, 11]), type: torch.float32


In [ ]:
# plt.imshow(obs[0][0], cmap="gray")
# plt.show()

In [ ]:
input_ = torch.randn(2, 17, 7, 11)
print(f"size: {input_.size()}, type: {input_.dtype}")

In [ ]:
net = env_module.GeeseNet()

In [ ]:
# Disable trainable for pretrained weights.
# for param in net.geese_net.parameters():
#     param.requires_grad = False

In [ ]:
out = net(obs_)
# print(f"size: {out.size()}, type: {out.dtype}")
print(f"size: {out['policy'].size()}, type: {out['policy'].dtype}")
print(f"size: {out['value'].size()}, type: {out['value'].dtype}")

In [ ]:
out

In [ ]:
# pytorch_total_params
params = sum(p.numel() for p in net.parameters())
print(f"{params:,}")

In [ ]:
# pytorch_total_params (trainable)
params = sum(p.numel() for p in net.parameters() if p.requires_grad)
print(f"{params:,}")

In [ ]:
net